In [ ]:
!echo $CUDA_VISIBLE_DEVICES

In [ ]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("TR2EN.txt", sep="\t").astype(str)

In [ ]:
df

In [ ]:
data = []
for item in df[:10500].iterrows():
    data.append(["translate english to turkish", item[1].EN, item[1].TR])

In [ ]:
df = pd.DataFrame(data, columns=["prefix", "input_text", "target_text"])

In [ ]:
train_df = df[:50]
eval_df = df[50:100]

In [ ]:
train_df

In [ ]:
import logging
import pandas as pd
from simpletransformers.t5 import T5Model, T5Args

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [ ]:
model_args = T5Args()
model_args.max_seq_length = 96
model_args.train_batch_size = 20
model_args.eval_batch_size = 20
model_args.num_train_epochs = 1
model_args.evaluate_during_training = True
model_args.evaluate_during_training_steps = 30000
model_args.use_multiprocessing = False
model_args.fp16 = False
model_args.save_steps = -1
model_args.save_eval_checkpoints = False
model_args.no_cache = True
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.preprocess_inputs = False
model_args.num_return_sequences = 1
model_args.wandb_project = "MT5 English-Turkish Translation"

model = T5Model("mt5", "google/mt5-small", args=model_args, use_cuda=False)

In [ ]:
model.train_model(train_df, eval_data=eval_df)

In [ ]:
model_args = T5Args()
model_args.max_length = 512
model_args.length_penalty = 1
model_args.num_beams = 10

In [ ]:
model = T5Model("mt5", "outputs", args=model_args, use_cuda=False)